In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7568,application_1589299642358_2063,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_timetable"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7563,application_1589299642358_2058,pyspark,idle,Link,Link,
7565,application_1589299642358_2060,pyspark,idle,Link,Link,
7567,application_1589299642358_2062,pyspark,idle,Link,Link,
7568,application_1589299642358_2063,pyspark,idle,Link,Link,✔


In [39]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
stop_time.printSchema()

In [41]:
stop_time.show(10)

An error was encountered:
Session 7568 did not reach idle status in time. Current status is busy.


In [42]:
@f.udf
def parent_station_id(stop_id):
    return stop_id.split(':')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .withColumn('stop_id', parent_station_id(stop_time.stop_id))
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00')
    )
    
)

An error was encountered:
Session 7568 did not reach idle status in time. Current status is busy.


In [ ]:
stop_time.show(10)

In [ ]:
stop_time.agg({'arrival_time': 'max', 'departure_time': 'max'}).show()

In [ ]:
departure = stop_time.alias('departure')
arrival = stop_time.withColumn('stop_sequence', stop_time.stop_sequence - 1).alias('arrival')

In [ ]:
timetable = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
    .select(
        departure.trip_id,
        departure.stop_sequence,
        departure.stop_id.alias('start_id'),
        departure.departure_time.alias('start_time'),
        arrival.arrival_time.alias('stop_time'),
        arrival.stop_id.alias('stop_id')
    )
)

In [ ]:
timetable.write.format("orc").save("/user/gflueck/timetable.orc", mode='overwrite')